# CH01.4. **Data Pipeline**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [ ]:
SEED_NUM = 2025
BATCH_SIZE = 50

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [ ]:
#(1) Import libraries
import os
import random
import numpy as np
import torch
import torchvision

#(2) Set up options
os.environ['PYTHONHASHSEED'] = str(SEED_NUM)
random.seed(a=SEED_NUM)
np.random.seed(seed=SEED_NUM)
torch.use_deterministic_algorithms(mode=True)
torch.manual_seed(seed=SEED_NUM)
torch.mps.manual_seed(seed=SEED_NUM)

#### 00.2. **사용자정의함수 정의**

In [3]:
pass

#### 00.3. **클래스 정의**

In [4]:
pass

<b></b>

## 01. **데이터 전처리 클래스**

#### 01.1. **데이터셋(Dataset)**

##### (1) **정의** : 대규모의 데이터를 효율적으로 처리하기 위한 데이터 인터페이스

##### (2) **특징** : 
##### $ \hspace{0.15cm} $ ① `__len__` 및 `__getitem__` 메서드를 구현하여 데이터 로더 클래스와 연동해 배치 처리, 셔플, 병렬 데이터 로딩 등의 기능을 손쉽게 활용
##### $ \hspace{0.15cm} $ ② `__getitem__` 메서드에서 이미지 리사이징, 정규화 등 다양한 전처리 과정을 쉽게 적용
##### $ \hspace{0.15cm} $ ③ 데이터가 실제로 필요할 때 불러오는 방식을 지원하여 메모리를 효율적으로 관리 가능 (Lazy loading)

#### 01.2. **데이터 로더(Data Loader)**

##### (1) **정의** : 모델 학습에 필요한 데이터의 전처리 및 학습 처리(배치, 셔플 등)을 지원하는 인터페이스

##### (2) **특징** : 
##### $ \hspace{0.15cm} $ ① 준비된 데이터는 큐(Queue;선입선출의 자료구조)를 통해 메인 프로세스에 전달
##### $ \hspace{0.15cm} $ ② 생성자(generator) 클래스로 정의되며, `next()` 메서드를 통해 차례로 값에 접근하기 때문에 메모리를 효율적으로 관리함 
##### $ \hspace{0.15cm} $ ③ `pin_memory`, `drop_last`, `sampler` 파라미터 등 다양한 옵션을 통해 세밀한 제어 가능

<b></b>

## 02. **데이터 불러오기 및 전처리**

#### 02.1. **이미지 전처리 파이프라인 정의** : `torchvision.transforms.Compose`

In [5]:
img_tf = torchvision.transforms.Compose(
    transforms=[
        torchvision.transforms.Resize(size=(16, 16)),
        torchvision.transforms.ToTensor()
    ]
)

#### **(`PLUS`)** 대표적인 이미지 전처리 함수들
#### $ \hspace{0.15cm} \cdot{} $ `torchvision.transforms.Resize()` 
#### $ \hspace{0.15cm} \cdot{} $ `RandomAffine()` : 아핀 변환(회전, 이동, 확대/축소, 전단 등) 적용
#### $ \hspace{0.15cm} \cdot{} $ `torchvision.transforms.Grayscale()` : 컬러 이미지를 흑백(grayscale) 이미지로 변환
#### $ \hspace{0.15cm} \cdot{} $ `torchvision.transforms.Normalize()` : 채널 별 이미지 정규화
#### $ \hspace{0.15cm} \cdots{} $

#### 02.2. **내장 데이터셋 로드 및 전처리**

In [6]:
#(1) Download train dataset
train = torchvision.datasets.CIFAR10(root='../../data', train=True, download=True, transform=img_tf)

In [ ]:
#(2) Print train dataset length
len(train)

50000

In [ ]:
#(3) Print image shape
train[0][0].size()

torch.Size([3, 16, 16])

#### **(`PLUS`)** Dataset 클래스의 인덱스 구조 : 
#### $ \hspace{0.15cm} \cdot{} $ `Dataset[num][0]` : `num`번 째 데이터셋의 입력 데이터
#### $ \hspace{0.15cm} \cdot{} $ `Dataset[num][1]` : `num`번 째 데이터셋의 타겟 데이터

#### 02.3. **데이터로더 객체로 변환**

In [9]:
#(1) Define `train_loader`
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=True)

#(2) Check batch count
len(train_loader)

1000

#### **(`PLUS`)** `BATCH_SIZE` 값($ 50 $)에 맞추어 나누어 $ \, 1000 $ 개로 구성됨

#### 02.4. **데이터로더 접근**

In [10]:
#(1) Define `inputs` and `targets`
inputs, targets = next(iter(train_loader))

#(2) Print `targets`
targets

tensor([0, 1, 7, 9, 3, 1, 7, 3, 8, 3, 3, 0, 7, 0, 7, 2, 9, 4, 4, 5, 8, 7, 8, 1,
        4, 6, 5, 1, 9, 0, 0, 0, 0, 7, 9, 7, 4, 2, 9, 7, 3, 3, 3, 3, 7, 9, 0, 4,
        6, 3])

#### 02.5. **배치 단위 반복(iteration) 시행**

In [11]:
for inputs, targets in train_loader :
    pass # anything #